### Exp: 2 Cluster customers first... Then map each cluster to vehicle

Assumption: Customer comes in cluster big small medium with some noise(somebody crossing da road) clustering them might mean they comes from the same vehicle (or come together from somewhere)

Cross : Create a HDBSCAN cluster then tune it accordingly

Cross OUTPUT : 
- df_cross['cluster'] : the cluster that the that customer belongs to with 
- cluster_cross : the cluster info of cross containin 
    - xy mid mean, min , max - timestamp (start stop avg) and COUNT of cluster size


camera used : mbk-14-12-vehicle 

### Load

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger import Cluster

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

pd.set_option('display.precision', 10)
CURRENT_CAM = "mbk-14-12"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)
# print(df_vehicle.to_string())
# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT]].to_string())

      is_plate_valid    travel_distance  vehicle_type_confidence         lifetime  ymin plate_number  xmin  province_confidence similarized_plate_number  plate_number_confidence  xmax                                                                                      image_name purchasing_power province             camera  travel_displacement            timestamp   car_brand  travel_trajectory  frame_no               original_video_name  ymax  plate_color_confidence vehicle_type  car_brand_confidence plate_color  plate_image_name       timestamp_precise           true_video_name                        video_name    xmid   ymid  timestamp_unix  timestamp_unix_end
2315            True    2828.2868920026             0.3558781743    14.2359998226     0       327722  1221         0.9515497684                   327722             0.9935601015  2191   2024-04-28T09-27-41_FLAG_P_FID_1291_OID_53g0_mbk-14-12-vehicle_1714271101_1714271261_1291.jpg          economy    th-10  mbk-14-12-vehicle    

### Preprocess

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

In [3]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

### Clustering the customer

In [4]:
df_cross ,  cluster_cross = Cluster.perform_cross_clustering(df_cross=df_cross)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT, 'cluster']].to_string())

In [5]:
print(Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST].to_string())

           timestamp_min        timestamp_max  count  cluster_id        xmid_mean       ymid_mean
1    2024-04-28 02:00:08  2024-04-28 02:25:24      5           0   505.3000000000  762.0000000000
0    2024-04-28 02:11:31  2024-04-28 15:31:18     75          -1  1630.7133333333  368.9800000000
21   2024-04-28 02:25:10  2024-04-28 02:25:27      2          20  2136.5000000000   29.0000000000
22   2024-04-28 02:27:18  2024-04-28 02:27:24      3          21  2107.6666666667  105.3333333333
16   2024-04-28 02:34:14  2024-04-28 02:34:21      3          15  2133.1666666667   58.6666666667
13   2024-04-28 02:44:29  2024-04-28 02:48:43      3          12  1876.3333333333  261.8333333333
61   2024-04-28 03:03:14  2024-04-28 03:03:56      2          60   965.0000000000  806.0000000000
105  2024-04-28 03:03:20  2024-04-28 03:03:29      2         104  1089.7500000000  735.5000000000
87   2024-04-28 03:03:25  2024-04-28 03:03:36      2          86  1044.0000000000  678.0000000000
104  2024-04-28 03:0

In [6]:
john =  Cluster.assign_cluster_to_vehicle(df_vehicle, cluster_cross)
print(john[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT_CLUSTERED]].to_string())

[]
[]
[]
[]
[]
[]
[2154, 2147]
[1021]
[2105, 2106]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[717]
[]
[]
[]
[2315]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[234]
[]
[]
[2241]
[]
[2104, 2105, 2106]
[]
[]
[]
[]
[]
[1610]
[]
[1655]
[]
[]
[]
[]
[]
[2164]
[1795]
[]
[]
[]
[913]
[]
[2104, 2105, 2106]
[2104, 2105, 2106]
[]
[1158]
[1158]
[]
[]
[1260, 1223]
[1049]
[]
[]
[1877]
[]
[]
[]
[2182]
[]
[]
[1083, 1084]
[]
[1049, 1020]
[]
[1059]
[]
[]
[]
[2182]
[]
[]
[]
[]
[1221, 1222]
[1221, 1222, 1223]
[1083, 1084]
[1083, 1084]
[]
[2121]
[]
[]
[]
[]
[1012]
[]
[]
[]
[]
[1702]
[]
[]
[1407]
[1504, 1480]
[]
[1175]
[1175]
[1012]
[2121]
[2121]
[2121]
[]
[]
[1772]
[1772]
[1655]
[1658]
[]
[1578]
[1578]
[]
[1409]
[1387]
[1329, 1330]
[1329, 1330]
[]
[1158]
[1158]
[]
[1877, 1878]
[1795]
[1517]
[]
[2121]
[]
[1558, 1559]
[1558, 1559]
[]
[]
[]
[]
[]
[1877, 1878, 1879]
[1558, 1559]
[1558, 1559]
[]
[]
[]
[1877]
[1845]
[1845]
[1795]
[1733]
[1480, 1478]
[]
[]
[]
[]
[]
[]
[]
[1407, 1455]
[1407]
[1795]
[1795]
[1795]
[1795]
[]
[]
[1504]
[]

TODO: find a better metric function to calculate distances